In [5]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import timedelta
import requests
from zipfile import ZipFile

def download_and_extract(zip_urls, download_folder, extract_folder):
    """
    Downloads ZIP files from provided URLs and extracts their contents.
    """
    os.makedirs(download_folder, exist_ok=True)
    os.makedirs(extract_folder, exist_ok=True)
    
    for idx, url in enumerate(zip_urls):
        print(f"({idx + 1}/{len(zip_urls)}) Downloading: {url}")
        zip_filename = os.path.join(download_folder, os.path.basename(url))
        
        # Download the file
        with requests.get(url, stream=True) as response:
            response.raise_for_status()
            with open(zip_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
        
        print(f"Downloaded: {zip_filename}")
        
        # Extract the ZIP file
        with ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
            print(f"Extracted: {zip_filename} to {extract_folder}")


def process_large_csv(csv_files, temp_folder, chunk_size=100000):
    """Reads large CSV files and splits them into individual MMSI files."""
    os.makedirs(temp_folder, exist_ok=True)
    mmsi_dict = {}

    for csv_file in csv_files:
        print(f"Processing {csv_file}...")
        with tqdm(total=sum(1 for _ in open(csv_file)) - 1, desc="Processing chunks", unit="rows") as pbar:
            for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
                # Filter out rows with 'Unknown value' in Navigational Status
                chunk = chunk[chunk['Navigational status'] != 'Unknown value']
                for mmsi, group in chunk.groupby('MMSI'):
                    if mmsi not in mmsi_dict:
                        mmsi_dict[mmsi] = group
                    else:
                        mmsi_dict[mmsi] = pd.concat([mmsi_dict[mmsi], group])
                pbar.update(len(chunk))
    
    # Save individual MMSI files
    for mmsi, group in tqdm(mmsi_dict.items(), desc="Saving MMSI files"):
        output_file = os.path.join(temp_folder, f"{mmsi}.csv")
        group.to_csv(output_file, index=False)

def filter_by_area(input_folder, output_folder, area_bounds):
    """Filters MMSI files for entries within a specified geographical area."""
    os.makedirs(output_folder, exist_ok=True)
    min_lat, max_lat, min_lon, max_lon = area_bounds

    for file in tqdm(os.listdir(input_folder), desc="Filtering by area"):
        if not file.endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(input_folder, file))
        df = df[(df['Latitude'] >= min_lat) & (df['Latitude'] <= max_lat) & 
                (df['Longitude'] >= min_lon) & (df['Longitude'] <= max_lon)]
        if len(df) >= 10:  # Ensure at least 10 consecutive entries
            df.to_csv(os.path.join(output_folder, file), index=False)

def extract_30min_sets(input_folder, output_folder):
    """Extracts 30-minute sets with consistent Navigational Status and saves each as a separate file with time range in the filename."""
    os.makedirs(output_folder, exist_ok=True)

    for file in tqdm(os.listdir(input_folder), desc="Extracting 30-min sets"):
        if not file.endswith('.csv'):
            continue

        # Load the CSV file
        file_path = os.path.join(input_folder, file)
        try:
            df = pd.read_csv(file_path)

            # Strip leading and trailing spaces and remove the '#' from column names
            df.columns = df.columns.str.strip().str.lstrip('# ')
            print(f"Cleaned column names: {list(df.columns)}")

            # Ensure the 'Timestamp' column exists
            if 'Timestamp' not in df.columns:
                print(f"Error: 'Timestamp' column not found in {file}")
                continue

            # Convert the 'Timestamp' column to datetime
            try:
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
            except Exception as e:
                print(f"Error parsing 'Timestamp' in {file}: {e}")
                continue

            # Drop rows with invalid timestamps
            df = df.dropna(subset=['Timestamp'])

            # Sort the DataFrame by timestamp
            df = df.sort_values('Timestamp')

            # Group into 30-minute intervals
            df['Time_Group'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds() // (30 * 60)
            grouped = df.groupby('Time_Group')

            # Process each 30-minute group
            valid_sets = 0
            for group_id, group in grouped:
                # Check if the Navigational Status is consistent
                if group['Navigational status'].nunique() == 1:
                    valid_sets += 1

                    # Get start and end times for the 30-minute interval
                    start_time = group['Timestamp'].min().strftime('%Y%m%d_%H%M')
                    end_time = group['Timestamp'].max().strftime('%Y%m%d_%H%M')

                    # Construct the output filename with time range in the format: original_filename_starttime-endtime.csv
                    output_filename = f"{os.path.splitext(file)[0]}_{start_time}-{end_time}.csv"
                    output_path = os.path.join(output_folder, output_filename)

                    # Save the 30-minute group as a separate file
                    group.to_csv(output_path, index=False)
                    print(f"Saved 30-min set: {output_filename}")

            if valid_sets == 0:
                print(f"No valid 30-min sets found in {file}.")

        except Exception as e:
            print(f"Error processing file {file}: {e}")

def analyze_navigational_status(input_folder):
    """Analyzes the distribution of 30-minute sets by Navigational Status."""
    status_counts = {}

    for file in tqdm(os.listdir(input_folder), desc="Analyzing Navigational Status"):
        if not file.endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(input_folder, file))
        status = df['Navigational status'].iloc[0]  # Assuming consistent status in each set
        status_counts[status] = status_counts.get(status, 0) + 1

    return status_counts

def integrate_pipeline_with_web_sources(zip_urls, temp_folder, area_bounds):
    """
    Downloads and processes ZIP files, extracting CSVs and running the pipeline.
    """
    download_folder = './downloaded_zips'
    extract_folder = './extracted_files'

    # Step 1: Download and Extract
    # download_and_extract(zip_urls, download_folder, extract_folder)

    # Step 2: Split CSVs into MMSI-specific files
    # process_large_csv([os.path.join(extract_folder, f) for f in os.listdir(extract_folder) if f.endswith('.csv')], temp_folder)

    # Step 3: Filter files by geographical area
    filtered_folder = './filtered_area'
    # filter_by_area(temp_folder, filtered_folder, area_bounds)

    # Step 4: Extract 30-minute sets
    sets_folder = './30min_sets'
    extract_30min_sets(filtered_folder, sets_folder)

    # Step 5: Analyze Navigational Status
    status_distribution = analyze_navigational_status(sets_folder)
    print("Distribution of Navigational Status:", status_distribution)

# Example URLs for ZIP files (replace with actual URLs)
zip_urls = [
    "https://web.ais.dk/aisdata/aisdk-2024-11-30.zip",
    "https://web.ais.dk/aisdata/aisdk-2024-11-29.zip",
    "https://web.ais.dk/aisdata/aisdk-2024-11-28.zip"
]

# Example usage
area_bounds = (55.0, 56.0, 7.0, 8.0)  # Define your area
temp_folder = './mmsi_temp'
integrate_pipeline_with_web_sources(zip_urls, temp_folder, area_bounds)


Extracting 30-min sets:   2%|▏         | 3/122 [00:00<00:06, 19.79it/s]

Cleaned column names: ['Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']
Saved 30-min set: 209715000_20241129_0201-20241129_0231.csv
Saved 30-min set: 209715000_20241129_0231-20241129_0236.csv
Saved 30-min set: 209715000_20241129_0315-20241129_0331.csv
Saved 30-min set: 209715000_20241129_0331-20241129_0401.csv
Saved 30-min set: 209715000_20241129_0401-20241129_0431.csv
Saved 30-min set: 209715000_20241129_0431-20241129_0454.csv
Cleaned column names: ['Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', '

Extracting 30-min sets:   5%|▍         | 6/122 [00:00<00:06, 18.05it/s]

Saved 30-min set: 211100000_20241128_1157-20241128_1227.csv
Saved 30-min set: 211100000_20241128_1227-20241128_1257.csv
Saved 30-min set: 211100000_20241128_1257-20241128_1327.csv
Saved 30-min set: 211100000_20241128_1327-20241128_1341.csv
Cleaned column names: ['Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']
Saved 30-min set: 211141000_20241128_0738-20241128_0808.csv
Saved 30-min set: 211141000_20241128_0808-20241128_0838.csv
Saved 30-min set: 211141000_20241128_0839-20241128_0908.csv
Saved 30-min set: 211141000_20241128_0908-20241128_0938.csv
Saved 30-min set: 211141000_20241128_0938-20241128_1008.csv
Saved 30-min set: 211141000_20241128_1008-20241128_1038.csv
Saved 30-min set: 211141000_20241128_1038-20241128_1108.csv
Saved 30-min set: 

Extracting 30-min sets:   7%|▋         | 9/122 [00:00<00:05, 20.57it/s]

...

Cleaned column names: ['Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']
Saved 30-min set: 374689000_20241128_1421-20241128_1450.csv
Saved 30-min set: 374689000_20241128_1451-20241128_1520.csv
Saved 30-min set: 374689000_20241128_1521-20241128_1549.csv
Saved 30-min set: 374689000_20241128_1554-20241128_1614.csv
Saved 30-min set: 374689000_20241128_1621-20241128_1650.csv
Saved 30-min set: 374689000_20241128_1651-20241128_1720.csv
Saved 30-min set: 374689000_20241128_1721-20241128_1750.csv
Saved 30-min set: 374689000_20241128_1753-20241128_1820.csv
Saved 30-min set: 374689000_20241128_1821-20241128_1850.csv
Saved 30-min set: 374689000_20241128_1851-20241128_1920.csv
Saved 30-min set: 374689000_20241128_1923-20241128_1948.csv
Saved 30-min set: 

Extracting 30-min sets: 100%|██████████| 122/122 [00:10<00:00, 11.28it/s]


Saved 30-min set: 636022640_20241129_0007-20241129_0037.csv
Saved 30-min set: 636022640_20241129_0038-20241129_0107.csv
Saved 30-min set: 636022640_20241129_0108-20241129_0137.csv
Saved 30-min set: 636022640_20241129_0138-20241129_0156.csv


Analyzing Navigational Status: 100%|██████████| 1563/1563 [00:06<00:00, 228.91it/s]

Distribution of Navigational Status: {'Under way using engine': 995, 'Engaged in fishing': 175, 'Restricted maneuverability': 274, 'Moored': 102, 'Under way sailing': 17}
